In [ ]:
USE_COLAB = False

In [ ]:
curren_dir = '/content/drive/MyDrive/path/to/folder/'

In [ ]:
import sys
if USE_COLAB:
    from google.colab import drive
    drive.mount('/content/drive/')
    sys.path.append(curren_dir) ## Place correct
else:
    sys.path.append('')

import numpy as np
import cv2
import os
from datetime import datetime

from PIL import Image

import torch
import torchvision

from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor, FasterRCNN_MobileNet_V3_Large_FPN_Weights
from torch.utils.data import Dataset, DataLoader, Subset
import torchvision.transforms.functional as F

from sklearn.model_selection import train_test_split
from tqdm import tqdm

from matplotlib import pyplot as plt
import pickle

import pandas as pd
import glob as glob
import random

In [ ]:
import matplotlib.patches as patches

def plot_image(img, boxes, scores, labels, dataset, save_path=None):

  '''
  Function that draws the BBoxes, scores, and labels on the image.

  inputs:
    img: input-image as numpy.array (shape: [H, W, C])
    boxes: list of bounding boxes (Format [N, 4] => N times [xmin, ymin, xmax, ymax])
    scores: list of conf-scores (Format [N] => N times confidence-score between 0 and 1)
    labels: list of class-prediction (Format [N] => N times an number between 0 and _num_classes-1)
    dataset: list of all classes e.g. ["background", "class1", "class2", ..., "classN"] => Format [N_classes]
  '''

  cmap = plt.get_cmap("tab20b")
  class_labels = np.array(dataset)
  colors = [cmap(i) for i in np.linspace(0, 1, len(class_labels))]
  height, width, _ = img.shape
  # Create figure and axes
  fig, ax = plt.subplots(1, figsize=(16, 8))
  # Display the image
  ax.imshow(img)

  for i, box in enumerate(boxes):
    class_pred = labels[i]
    conf = scores[i]
    width = box[2] - box[0]
    height = box[3] - box[1]
    rect = patches.Rectangle(
        (box[0], box[1]),
        width,
        height,
        linewidth=2,
        edgecolor=colors[int(class_pred)],
        facecolor="none",
    )
    # Add the patch to the Axes
    ax.add_patch(rect)
    plt.text(
        box[0], box[1],
        s=class_labels[int(class_pred)] + " " + str(int(100*conf)) + "%",
        color="white",
        verticalalignment="top",
        bbox={"color": colors[int(class_pred)], "pad": 0},
    )

  # Used to save inference phase results
  if save_path is not None:
    plt.savefig(save_path)

  plt.show()

In [ ]:
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor, FasterRCNN_MobileNet_V3_Large_FPN_Weights
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor, FasterRCNN_MobileNet_V3_Large_FPN_Weights

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print("We use the following device: ", device)
# load a model; pre-trained on COCO
# model = torchvision.models.detection.fasterrcnn_mobilenet_v3_large_fpn(weights='COCO_V1').to(device)
# model = torchvision.models.detection.fasterrcnn_resnet50_fpn_v2(weights="COCO_V1").to(device)
model = torchvision.models.detection.fasterrcnn_mobilenet_v3_large_fpn(weights=FasterRCNN_MobileNet_V3_Large_FPN_Weights.COCO_V1)

In [ ]:
def img_transform(img):
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB).astype(np.float32)
  img /= 255.0
  img = torch.from_numpy(img).permute(2,0,1)
  return img

## Train

In [ ]:
# # Import dataset

# !pip install roboflow

# from roboflow import Roboflow
# rf = Roboflow(api_key="hHft0q8E9j8NLCGwL0rE")
# project = rf.workspace("nora-slimani").project("trash-detection-otdmj")
# dataset = project.version(35).download("coco")

In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="hHft0q8E9j8NLCGwL0rE")
project = rf.workspace("trash-dataset-for-oriented-bounded-box").project("trash-detection-1fjjc")
version = project.version(14)
dataset = version.download("coco")


In [ ]:
# importare il file 'train_images_info.csv'
train_df = pd.read_csv(r'/content/train_images_info.csv')
train_df.head()

In [ ]:
train_df["x1"] = pd.to_numeric(train_df["x1"])
train_df["y1"] = pd.to_numeric(train_df["y1"])
train_df["w"] = pd.to_numeric(train_df["w"])
train_df["h"] = pd.to_numeric(train_df["h"])

In [ ]:
train_df["x2"] = train_df["x1"] + train_df["w"]
train_df["y2"] = train_df["y1"] + train_df["h"]

In [ ]:
train_df.drop(columns=['w', 'h', 'height', 'width'], inplace = True)
train_df = train_df[['ann_id', 'image_id', 'category_id', 'x1', 'y1', 'x2', 'y2', 'file_name']]
train_df.head()

## Validation

In [ ]:
# importare il file 'train_images_info.csv'
valid_df = pd.read_csv(r'/content/valid_images_info.csv')
valid_df.head()

In [ ]:
valid_df["x1"] = pd.to_numeric(valid_df["x1"])
valid_df["y1"] = pd.to_numeric(valid_df["y1"])
valid_df["w"] = pd.to_numeric(valid_df["w"])
valid_df["h"] = pd.to_numeric(valid_df["h"])

In [ ]:
valid_df["x2"] = valid_df["x1"] + valid_df["w"]
valid_df["y2"] = valid_df["y1"] + valid_df["h"]

In [ ]:
valid_df.drop(columns=['w', 'h', 'height', 'width'], inplace = True)
valid_df = valid_df[['ann_id', 'image_id', 'category_id', 'x1', 'y1', 'x2', 'y2', 'file_name']]
valid_df.head()

## Test

In [ ]:
# importare il file 'train_images_info.csv'
test_df = pd.read_csv(r'/content/test_images_info.csv')
test_df.head()

In [ ]:
test_df["x1"] = pd.to_numeric(test_df["x1"])
test_df["y1"] = pd.to_numeric(test_df["y1"])
test_df["w"] = pd.to_numeric(test_df["w"])
test_df["h"] = pd.to_numeric(test_df["h"])

In [ ]:
test_df["x2"] = test_df["x1"] + test_df["w"]
test_df["y2"] = test_df["y1"] + test_df["h"]

In [ ]:
test_df.drop(columns=['w', 'h', 'height', 'width'], inplace = True)
test_df = test_df[['ann_id', 'image_id', 'category_id', 'x1', 'y1', 'x2', 'y2', 'file_name']]
test_df.head()

## Dataset

In [ ]:
# scrivo train_df in file train_df.csv
train_df.to_csv("/content/train_df.csv")
valid_df.to_csv("/content/validation_df.csv")
test_df.to_csv("/content/test_df.csv")

In [ ]:
# path_csv = sys.path[-1] + 'labels_crowdai.csv'
# img_dir = 'object-detection-crowdai/'
path_csv = "/content/train_df.csv"
# img_dir = "/content/trash-detection-35/train/"
img_dir = "/content/Trash-Detection-14/train/"

path_csv_valid = "/content/validation_df.csv"
# img_dir_test = "/content/trash-detection-35/valid/"
img_dir_valid = "/content/Trash-Detection-14/valid/"

path_csv_test = "/content/test_df.csv"
img_dir_test = "/content/Trash-Detection-14/test/"

In [ ]:
df = pd.read_csv(path_csv)
print("Number of total bounding boxes: ", len(df))

image_paths = glob.glob(f"{img_dir}/*.jpg")
all_images = [image_path.split(os.path.sep)[-1] for image_path in image_paths]
all_images = sorted(all_images)
print("Number of unique images: ", len(all_images))

In [ ]:
df_valid = pd.read_csv(path_csv_valid)
print("Number of total bounding boxes: ", len(df_valid))

image_paths_valid = glob.glob(f"{img_dir_valid}/*.jpg")
all_images_valid = [image_path_valid.split(os.path.sep)[-1] for image_path_valid in image_paths_valid]
all_images_valid = sorted(all_images_valid)
print("Number of unique images: ", len(all_images_valid))

In [ ]:
df_test = pd.read_csv(path_csv_test)
print("Number of total bounding boxes: ", len(df_test))

image_paths_test = glob.glob(f"{img_dir_test}/*.jpg")
all_images_test = [image_path_test.split(os.path.sep)[-1] for image_path_test in image_paths_test]
all_images_test = sorted(all_images_test)
print("Number of unique images: ", len(all_images_test))

## Dataset creation

In [ ]:
'''
Class that holds all the augmentation related attributes
'''
class Transformation():
    # This provides a random probability of the augmentation to be applied or not
    def get_probability(self):
        return np.random.choice([False, True], replace=False, p=[0.5, 0.5])

    # Increases the contrast by a factor of 2
    def random_adjust_contrast(self, image, enable=None):
        enable = self.get_probability() if enable is None else enable
        return F.adjust_contrast(image, 2) if enable else image

    # Increaes the brightness by a factor of 2
    def random_adjust_brightness(self, image, enable=None):
        enable = enable = self.get_probability() if enable is None else enable
        return F.adjust_brightness(image,2) if enable else image

    # Horizontal flip
    def random_hflip(self, image, boxes, enable=None):
        enable = enable = self.get_probability() if enable is None else enable
        if enable:
          #flip image
          new_image = F.hflip(image)

          #flip boxes
          new_boxes = boxes.clone()
          new_boxes[:, 0] = image.shape[2] - boxes[:, 0]  # image width - xmin
          new_boxes[:, 2] = image.shape[2] - boxes[:, 2]  # image_width - xmax
          new_boxes = new_boxes[:, [2, 1, 0, 3]]          # Interchange the xmin and xmax due to mirroring
          return new_image, new_boxes
        else:
          return image, boxes

In [ ]:
class vehicleDataset(Dataset):
  def __init__(self, img_path, label_path, classes, transforms=None):
    super().__init__()
    print("Preparing the dataset...")

    self.image_dir = img_path
    self.gt_info = pd.read_csv(label_path)
    self.classes = classes
    self.transforms = transforms

    # Create a list of image file names (in sorted order - this is optional)
    self.image_paths = glob.glob(f"{img_path}/*.jpg")
    all_images = [image_path.split(os.path.sep)[-1] for image_path in self.image_paths]
    self.all_images = sorted(all_images)

    images_to_keep = []

    id_to_name = {id_: name for id_, name in enumerate(self.classes)}

    # Map Label (str) --> Label (int)
    for i in range(len(self.gt_info)):
      label = self.gt_info.loc[i, 'category_id']
      label = id_to_name[label]
      if pd.notna(label):  # Check if label is not NaN
              self.gt_info.loc[i, 'category_id'] = self.classes.index(label)
              # Add the image name to the list of images to keep
              images_to_keep.append(self.gt_info.loc[i, 'file_name'])

    self.gt_info.loc[i, 'category_id'] = self.classes.index(label)

    # Filter the dataset based on given conditions:
    self.filter_dataset()
    print("Dataset prepared")

  def __getitem__(self, idx):
    target = {}

    # Read input image
    # image_name = self.all_images[idx]
    image_name = idx
    image_path = os.path.join(self.image_dir, image_name)
    image = cv2.imread(image_path)
    image = img_transform(image)

    # Fetch GT infos for given image
    gt_info = self.gt_info[self.gt_info['file_name'] == image_name]

    boxes = torch.Tensor(gt_info[['x1', 'y1', 'x2', 'y2']].values).float()
    labels = torch.LongTensor(gt_info['category_id'].values.tolist())

    if self.transforms:
        # image = self.transforms.random_adjust_contrast(image, enable=True)
        # image = self.transforms.random_adjust_brightness(image, enable=True)
        # image, boxes = self.transforms.random_hflip(image, boxes, enable=True)
        print("transforms")

    target["boxes"] = boxes     # Hint: Shape -> [N, 4] with N = Number of Boxes
    target["labels"] = labels   # Hint: Shape -> [N] with N = Number of Boxes

    return image, target

  def __len__(self):
    return len(self.all_images)

  def get_name(self, idx):
    image_name = self.all_images[idx]
    gt_info = self.gt_info[self.gt_info['file_name'] == image_name]
    return gt_info["file_name"]


  '''
  Filter the dataset by removing images with no labels and incorrect bounding boxes
  '''
  def filter_dataset(self):
    print("Filtering the dataset...")
    remove_images = []

    # There are no labels for some images because they show an 'empty' scene → these images should be filtered out.
    for image_file in self.all_images.copy():
      if image_file not in self.gt_info['file_name'].values:
        remove_images.append(image_file)
        self.all_images.remove(image_file)
    print("Images removed with no labels: ", len(remove_images))

    # There are incorrect bounding boxes in the dataset (e.g. xmax=xmin).
    valid_box_mask = (self.gt_info['x2'] - self.gt_info['x1'] > 0) & (self.gt_info['y2'] - self.gt_info['y1'] > 0)
    print("Images removed with incorrect bounding boxes: ", len(self.gt_info) - valid_box_mask.shape[0])
    self.gt_info = self.gt_info[valid_box_mask] # TODO: correct this masking

In [ ]:
# labels_classes = ["background","Aluminium foil","Bottle cap","Broken glass","Cigarette","Clear plastic bottle","Crisp packet","Cup",
#           "Drink can","Food Carton","Food container","Food waste","Garbage bag","Glass bottle","Lid","Other Carton","Other can",
#           "Other container","Other plastic","Other plastic bottle","Other plastic wrapper","Paper","Paper bag","Plastic bag wrapper",
#           "Plastic film","Pop tab","Single-use carrier bag","Straw","Styrofoam piece","Unlabeled litter"]

In [ ]:
labels_classes = ["Background","Glass","Metal","Paper","Plastic","Waste"]

In [ ]:
id_to_name = {id_: name for id_, name in enumerate(labels_classes)}

In [ ]:
dataset = vehicleDataset(img_dir_test, path_csv_test, labels_classes, transforms=Transformation())
num_images = 5
# indexes = []
# x = ["glass207_jpg.rf.578f30def39b5c7d69d004f459e54e30.jpg", "000050_jpg.rf.043db995836b9fcb08a484206fe7a8c6.jpg",
#      "paper132_jpg.rf.0008320c7b1e1857147763a80df50fe9.jpg", "000081_jpg.rf.0085f17f07c7ddf1f7b8790430fba450.jpg",
#      "000055_jpg.rf.0c6f1c63f72987cc8c896a56123bf993.jpg"]
x = ["glass252_jpg.rf.0588fd4e975a294f5062e7470e34dabe.jpg", "metal223_jpg.rf.0b435a8a304e807166bf2e4fa4c353ff.jpg",
     "paper220_jpg.rf.57167f8cca6f68de931682ad0ef679bd.jpg", "PET1-648_jpg.rf.585745955900675e2976a0128a16a780.jpg",
     "000007_jpg.rf.55298639ec0a0498f59e450e1ce69eb4.jpg"]
# for name in x:

#   print(name, valid_df[valid_df['file_name'] == name]['ann_id'])

# # x = [470, 5, 6, 0, 3, 40, 41, 42, 43, 44, 45]

for i in range(len(x)):
  img, target = dataset.__getitem__(x[i])
  img = img.cpu().permute(1,2,0).numpy()
  boxes = target['boxes'].numpy()
  labels = target['labels'].numpy()
  scores = [1]*len(labels)
  # info = list(dataset.get_name(x[i]))
  print("\nName Image: ", x[i])
  plot_image(img, boxes, scores, labels, labels_classes)

In [ ]:
dataset = vehicleDataset(img_dir, path_csv, labels_classes, transforms=Transformation()) # here your arguments
num_images = 5

for _ in range(num_images):
  x = random.randint(0, (dataset.__len__()-1))
  img, target = dataset.__getitem__(x)
  img = img.cpu().permute(1,2,0).numpy()
  boxes = target['boxes'].numpy()
  labels = target['labels'].numpy()
  scores = [1]*len(labels)
  info = list(dataset.get_name(x))
  print("\nName Image: ", info[0])
  plot_image(img, boxes, scores, labels, labels_classes)

## Fine tune

In [ ]:
# Hyperparameters

SEED = 42

TEST_SIZE = 0.2

NUM_EPOCHS = 10

LR = 0.005
LR_MOMENTUM=0.9
LR_DECAY_RATE=0.0005

LR_SCHED_STEP_SIZE = 0.1
LR_SCHED_GAMMA = 0.1

BATCH_SIZE = 32

NUM_TEST_IMAGES = 5
NMS_THRESH = 0.01

In [ ]:
current_dir = curren_dir if USE_COLAB else os.getcwd()

# Fetch current date and time
now = datetime.now()
# dt_string = now.strftime("%d-%m-%Y-%H-%M-%S")
dt_string = "images"
output_dir_name = "output-" + dt_string

OUTPUT_DIR = os.path.join(current_dir, output_dir_name)
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# model = torchvision.models.detection.fasterrcnn_resnet50_fpn(min_size=300, max_size=480, pretrained=True)
# model = torchvision.models.detection.fasterrcnn_resnet50_fpn_v2(min_size=300, max_size=480, weights="COCO_V1").to(device)
model = torchvision.models.detection.fasterrcnn_mobilenet_v3_large_fpn(weights=FasterRCNN_MobileNet_V3_Large_FPN_Weights.COCO_V1)

num_classes = len(labels_classes)  # 3 classes + background
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
model.to(device)

num_epochs = NUM_EPOCHS

# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=LR, momentum=LR_MOMENTUM, weight_decay=LR_DECAY_RATE)

# create a train- and validation-dataset with our vehicleDataset
# split the dataset in train and test set
# dataset = vehicleDataset(img_dir, path_csv, labels_classes, transforms=Transformation())      # here your arguments
# dataset_test = vehicleDataset(img_dir, path_csv, labels_classes, transforms=None)             # here your arguments

dataset = vehicleDataset(img_dir, path_csv, labels_classes, transforms=None)
dataset_valid = vehicleDataset(img_dir_valid, path_csv_valid, labels_classes, transforms=None)
dataset_test = vehicleDataset(img_dir_test, path_csv_test, labels_classes, transforms=None)

torch.manual_seed(SEED)
# indices = torch.randperm(len(dataset)).tolist()
indices = torch.randperm(len(dataset)).tolist()
indices_valid = torch.randperm(len(dataset_valid)).tolist()
indices_test = torch.randperm(len(dataset_test)).tolist()

#test_size = int(len(dataset) * TEST_SIZE)
# dataset = torch.utils.data.Subset(dataset, indices[:-test_size])
# dataset_test = torch.utils.data.Subset(dataset_test, indices[-test_size:])
dataset = torch.utils.data.Subset(dataset, indices)
dataset_valid = torch.utils.data.Subset(dataset_valid, indices_valid)
dataset_test = torch.utils.data.Subset(dataset_test, indices_test)

# create a learning rate scheduler
# TODO: step size to be tuned !
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, \
                                                step_size=LR_SCHED_STEP_SIZE, \
                                                gamma=LR_SCHED_GAMMA)

def collate_fn(batch):
  return tuple(zip(*batch))

# define training and validation data loaders
data_loader = torch.utils.data.DataLoader(
  dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2,
  collate_fn=collate_fn)

# data_loader_valid = torch.utils.data.DataLoader(
#   dataset_test, batch_size=1, shuffle=False, num_workers=2,
#   collate_fn=collate_fn)

data_loader_valid = torch.utils.data.DataLoader(
  dataset_valid, batch_size=1, shuffle=False, num_workers=2,
  collate_fn=collate_fn)

data_loader_test = torch.utils.data.DataLoader(
    dataset_test, batch_size = 1, shuffle=False, num_workers=2,
    collate_fn=collate_fn
)

In [ ]:
'''
Function to train the model over one epoch.
'''
def train_one_epoch(model, optimizer, data_loader, device, epoch):
  train_loss_list = []

  tqdm_bar = tqdm(data_loader, total=len(data_loader))
  for idx, data in enumerate(tqdm_bar):
    optimizer.zero_grad()
    images, targets = data

    images = list(image.to(device) for image in images)
    targets = [{k: v.to(device) for k, v in t.items()} for t in targets]  # targets = {'boxes'=tensor, 'labels'=tensor}

    losses = model(images, targets)

    loss = sum(loss for loss in losses.values())
    loss_val = loss.item()
    train_loss_list.append(loss.detach().cpu().numpy())

    loss.backward()
    optimizer.step()

    tqdm_bar.set_description(desc=f"Training Loss: {loss:.3f}")

  return train_loss_list

'''
Function to validate the model
'''
def evaluate(model, data_loader_test, device):
    val_loss_list = []

    tqdm_bar = tqdm(data_loader_test, total=len(data_loader_test))

    for i, data in enumerate(tqdm_bar):
        images, targets = data

        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        with torch.no_grad():
            losses = model(images, targets)

        loss = sum(loss for loss in losses.values())
        loss_val = loss.item()
        val_loss_list.append(loss_val)

        tqdm_bar.set_description(desc=f"Validation Loss: {loss:.4f}")
    return val_loss_list

'''
Function to plot training and valdiation losses and save them in `output_dir'
'''
def plot_loss(train_loss, valid_loss):
    figure_1, train_ax = plt.subplots()
    figure_2, valid_ax = plt.subplots()

    train_ax.plot(train_loss, color='blue')
    train_ax.set_xlabel('Iteration')
    train_ax.set_ylabel('Training Loss')

    valid_ax.plot(valid_loss, color='red')
    valid_ax.set_xlabel('Iteration')
    valid_ax.set_ylabel('Validation loss')

    figure_1.savefig(f"{OUTPUT_DIR}/train_loss.png")
    figure_2.savefig(f"{OUTPUT_DIR}/valid_loss.png")
    plt.close()

In [ ]:
# find latest saved chcekpoint
checkpoint_files = [f for f in os.listdir(OUTPUT_DIR) if f.endswith('.pth')]
if checkpoint_files:
    # Last ckpt file
    checkpoint_files.sort()
    latest_checkpoint = os.path.join(OUTPUT_DIR, checkpoint_files[-1])

    # Load the ckpt
    checkpoint = torch.load(latest_checkpoint)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    start_epoch = checkpoint['epoch']
    loss_dict = checkpoint['loss_dict']

    print("Keep running from last check-point")
else:
    start_epoch = 0
    loss_dict = {'train_loss': [], 'valid_loss': []}

'''
Train the model over all epochs
'''
for epoch in range(start_epoch, num_epochs):
  print("----------Epoch {}----------".format(epoch+1))

  # Train the model for one epoch
  train_loss_list = train_one_epoch(model, optimizer, data_loader, device, epoch)
  loss_dict['train_loss'].extend(train_loss_list)

  lr_scheduler.step()

  # Run evaluation
  valid_loss_list = evaluate(model, data_loader_valid, device)
  loss_dict['valid_loss'].extend(valid_loss_list)

  # Svae the model ckpt after every epoch
  ckpt_file_name = f"{OUTPUT_DIR}/epoch_{epoch+1}_model.pth"
  torch.save({
    'epoch': epoch+1,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'loss_dict': loss_dict
  }, ckpt_file_name)

  # NOTE: The losses are accumulated over all iterations
  plot_loss(loss_dict['train_loss'], loss_dict['valid_loss'])

# Store the losses after the training in a pickle
with open(f"{OUTPUT_DIR}/loss_dict.pkl", "wb") as file:
    pickle.dump(loss_dict, file)

print("Training Finished !")

In [ ]:
import shutil

def results(img_name, class_name, confidence, boxes):
  #<class_name> <confidence> <left> <top> <right> <bottom>
  string = ""
  path = "/content/results/"

  if not os.path.exists(path):
    print("Create directory")
    os.mkdir(path)

  # create file with file_name
  file_name = path + img_name[:-3] + "txt"

  for i in range(len(class_name)):
    # print("CLASS: ", class_name[i])
    # print("CONFIDENCE: ", confidence[i])
    # print("BOXES: ", str(boxes[i]))
    x1 = str(boxes[i][0])
    y1 = str(boxes[i][1])
    x2 = str(boxes[i][2])
    y2 = str(boxes[i][3])
    # string = str(labels_classes[class_name[i] + 1]) + " " + str(confidence[i]) + str(boxes[i])
    string += str(labels_classes[class_name[i]]) + " " + str(confidence[i]) + " " + x1 + " " + y1 + " " + x2 + " " + y2 + "\n"

  f = open(file_name, "a")
  f.write(string)
  f.close()

In [ ]:
def inference(img, img_name, model, detection_threshold=0.70):
  '''
  Infernece of a single input image

  inputs:
    img: input-image as torch.tensor (shape: [C, H, W])
    model: model for infernce (torch.nn.Module)
    detection_threshold: Confidence-threshold for NMS (default=0.7)

  returns:
    boxes: bounding boxes (Format [N, 4] => N times [xmin, ymin, xmax, ymax])
    labels: class-prediction (Format [N] => N times an number between 0 and _num_classes-1)
    scores: confidence-score (Format [N] => N times confidence-score between 0 and 1)
  '''
  model.eval()

  img = img.to(device)
  outputs = model([img])

  boxes = outputs[0]['boxes'].data.cpu().numpy()
  scores = outputs[0]['scores'].data.cpu().numpy()
  labels = outputs[0]['labels'].data.cpu().numpy()

  # print("Boxes: ", boxes)
  # print("Labels: ", labels)
  # print("Scores: ", scores)

  results(img_name, labels, scores, boxes)

  boxes = boxes[scores >= detection_threshold].astype(np.int32)
  labels = labels[scores >= detection_threshold]
  scores = scores[scores >= detection_threshold]

  return boxes, scores, labels

In [ ]:
d = vehicleDataset("/content/Trash-Detection-14/test", "/content/test_df.csv", labels_classes, transforms=Transformation()) # here your arguments

In [ ]:
!rm -rf /content/Trash-Detection-14/test/*.json

In [ ]:
random.seed(SEED)
num_images = 110

x = 0

for filename in os.listdir("/content/Trash-Detection-14/test/"):
# for _ in range(num_images):
  # x = random.randint(0, (d.__len__()-1))
  print("FILE NAME: ", filename)
  img, target = d.__getitem__(x)
  img = img.to(device)
  info = list(d.get_name(x))
  print("File name: ", info[0])

  # name_image = list(dataset_test.get_name(x))
  # print("Image: ", name_image)

  # Load last checkpoint
  # CHANGE THE OUTPUT_DIR IF CKPT IS STORED ELSEWHERE
  checkpoint_dir = f"/{OUTPUT_DIR}/epoch_5_model.pth"
  checkpoint = torch.load(checkpoint_dir, map_location=device)
  model.load_state_dict(checkpoint['model_state_dict'])

  boxes, scores, labels = inference(img, info[0], model)

  img = img.squeeze(0).cpu().permute(1,2,0).numpy()
  plot_image(img, boxes, scores, labels, labels_classes, save_path=f"{OUTPUT_DIR}/" + info[0])

  x += 1

prime 15 esecuzioni<br>
Training Loss: 0.134<br>
Validation Loss: 0.0716<br>
parameters: LR = 0.005
LR_MOMENTUM=0.9
LR_DECAY_RATE=0.0005


Training Loss: 0.022:
Validation Loss: 0.0506:

In [ ]:
#!zip -r output_resnet.zip /content/output-12-04-2024-07-08-06/*.png

In [ ]:
# from google.colab import files
# files.download("/content/output_resnet.zip")

In [ ]:
!mkdir /content/results_images/

In [ ]:
%cd /content/output-images
%cp *.jpg /content/results_images/

In [ ]:
%cd /content/

In [ ]:
!zip -r results.zip /content/results

In [ ]:
!zip -r results_images.zip /content/results_images/

In [ ]:
from google.colab import files
files.download("/content/results.zip")
files.download("/content/results_images.zip")